In [1]:
import sys
sys.path.append('..')
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from PreprocessText import read_doc, preprocess_text
import transformers
from transformers import BertModel, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from keyphrase_vectorizers import KeyphraseCountVectorizer
import torch
import re
import pickle
import numpy as np
from typing import List
import nltk
from nomic import embed

import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
if torch.cuda.is_available():
    # CUDA is available, you can proceed to use it
    device = torch.device('cuda')
    print('CUDA is available. Using GPU.')
else:
    # CUDA is not available, use CPU
    device = torch.device('cpu')
    print('CUDA is not available. Using CPU.')

CUDA is not available. Using CPU.


In [27]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)
vectorizer = KeyphraseCountVectorizer()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def remove_useless_info(patent):
    positions_to_remove = {2, 3, 4}
    patent = [element for index, element in enumerate(patent) if index not in positions_to_remove]
    return patent

def remove_allcaps(patent):
    patent = [element for index, element in enumerate(patent) if not element.replace(" ", "").isupper()]
    return patent

def remove_starting_brackets(patent):
    for index, element in enumerate(patent):
        if element.startswith('[') and re.match(r'\[\d+\]', element):
            # Use regular expression to remove brackets and digits
            processed_string = re.sub(r'\[\d+\]', '', element)
            patent[index] = processed_string
    return patent

def process_paragraphs(patent: List[str]):
    patent = remove_useless_info(patent)
    patent = remove_allcaps(patent)
    patent = remove_starting_brackets(patent)
    return patent

In [28]:
# path = r'C:\Users\johnt\Deep_Learning_Assignments\KeywordExtraction'

In [29]:
# Each patent is a big string, and all of the paragraphs are just separated by '\n'
# patents, labels = read_doc(r'C:\Users\johnt\Downloads\20240106021625210-AMD (2).docx')

# Split each document into its paragraphs by splitting on '\n',
# So now each doc is a list of strings
# Then process all of these paragraphs according to the process_paragraphs function above
# Pop the last element from the list because it is an empty string paragraph
# for i in range(len(patents)):
#     patents[i] = patents[i].split('\n')
#     patents[i] = process_paragraphs(patents[i])
#     patents[i].pop()

In [30]:
# with open('data.pkl', 'wb') as picklefile:
#     pickle.dump((patents, labels), picklefile)

In [31]:
with open('data.pkl', 'rb') as picklefile:
    loaded_data = pickle.load(picklefile)

In [32]:
patents = loaded_data[0]
labels = loaded_data[1]

In [33]:
def get_keywords(doc: List[str], doc_average=None, num_keywords=10):
    document_keyphrase_matrix = vectorizer.fit_transform(doc).toarray()
    keyphrases = vectorizer.get_feature_names_out()

    doc_tokens = [tokenizer(paragraph, return_tensors="pt", add_special_tokens=False, max_length=512, truncation=True) for paragraph in doc]
    keyphrase_tokens = [tokenizer(phrase, return_tensors="pt", add_special_tokens=False) for phrase in keyphrases]
    
    paragraph_embeddings_tensor, paragraph_embeddings_list = embed_paragraphs(doc_tokens)
    kp_embeddings_tensor, kp_embeddings_list = embed_keyphrases(keyphrase_tokens)
    
    if doc_average == "keyphrase":
        doc_embedding = embed_doc_by_keyphrase(paragraph_embeddings_tensor, kp_embeddings_tensor, kp_embeddings_list)
    else:
        doc_embedding = embed_doc_by_paragraph(paragraph_embeddings_tensor)
    
    if num_keywords > len(keyphrases):
        num_keywords = len(keyphrases)
        
    doc_kp_sim = cosine_similarity(doc_embedding, kp_embeddings_tensor)
    top_k_kp = np.argsort(doc_kp_sim)[0][::-1][:num_keywords]
    
    top_keyphrases = [keyphrases[x] for x in top_k_kp]
    
    return top_keyphrases

In [34]:
def embed_doc_by_keyphrase(paragraph_embeddings_tensor, kp_embeddings_tensor, kp_embeddings_list):
    cosine_similarities = cosine_similarity(paragraph_embeddings_tensor, kp_embeddings_tensor)
    paragraph_kw_indices = np.argmax(cosine_similarities, axis=1)
        
    paragraph_kw_embeddings = []
    for x in paragraph_kw_indices:
        paragraph_kw_embeddings.append(kp_embeddings_list[x])
        
    paragraph_kw_embeddings = torch.stack(paragraph_kw_embeddings, dim=0)
    paragraph_kw_embeddings = paragraph_kw_embeddings.squeeze(1)
    doc_embedding = paragraph_kw_embeddings.mean(dim=0)
    doc_embedding = doc_embedding.unsqueeze(0)
    return doc_embedding

In [35]:
def embed_doc_by_paragraph(paragraph_embeddings_tensor):
    doc_embedding = paragraph_embeddings_tensor.mean(axis=0)
    doc_embedding = doc_embedding.unsqueeze(0)
    return doc_embedding

In [36]:
def embed_paragraphs(doc_tokens: List[transformers.tokenization_utils_base.BatchEncoding]):
    paragraph_embeddings = []

    model.eval()
    with torch.no_grad():
        for i, token in enumerate(doc_tokens):
            outputs = model(**token)
            embeddings = outputs.last_hidden_state
            embeddings = embeddings.mean(dim=1)
            paragraph_embeddings.append(embeddings)

    paragraph_embeddings_stacked = torch.stack(paragraph_embeddings, dim=0)
    paragraph_embeddings_stacked = paragraph_embeddings_stacked.squeeze(1)
    
    return paragraph_embeddings_stacked, paragraph_embeddings

In [37]:
def embed_keyphrases(keyphrase_tokens: List[transformers.tokenization_utils_base.BatchEncoding]):
    kp_embeddings = []

    model.eval()
    with torch.no_grad():
        for token in keyphrase_tokens:
            kp_output = model(**token)
            kp_embedding = kp_output.last_hidden_state
            kp_embedding = kp_embedding.mean(dim=1)
            kp_embeddings.append(kp_embedding)

    kp_embeddings_stacked = torch.stack(kp_embeddings, dim=0)
    kp_embeddings_stacked = kp_embeddings_stacked.squeeze(1)
    
    return kp_embeddings_stacked, kp_embeddings

# Test functions

In [38]:
test = get_keywords(patents[0])

In [21]:
test

['å.',
 'gate dielectric layer',
 'gate dielectric material',
 'molybdenum',
 'polycrystalline silicon gate',
 'metal gate electrode',
 'best mode',
 'various obvious respects',
 'first polycrystalline silicon layer',
 'other embodiments']

In [14]:
label2pat = dict(zip(labels, patents))
patent_keyphrase = {}

i = 0
for label, patent in label2pat.items():
    print(i)
    i += 1
    keyphrases = get_keywords(patent)
    
    patent_keyphrase[label] = keyphrases

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [15]:
patent_df = pd.DataFrame(patent_keyphrase)

In [17]:
patent_df = patent_df.T

In [18]:
# If working on Linux, might need to change the '\' to '/' in the file name
patent_df.to_csv('.\patent_to_keyword.csv')

# Other models
#### This was just some very hacky testing on other potential models, didn't get very far

#### Check out this link for KW extraction models: https://huggingface.co/spaces/mteb/leaderboard

In [5]:
model_id = 'mixedbread-ai/mxbai-embed-large-v1'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

In [7]:
max_length = 4096

In [8]:
from sentence_transformers.util import cos_sim

# For retrieval you need to pass this prompt. Please find our more in our blog post.
def transform_query(query: str) -> str:
    """ For retrieval, add the prompt for query (not for documents).
    """
    return f'Represent this sentence for searching relevant passages: {query}'

# The model works really well with cls pooling (default) but also with mean poolin.
def pooling(outputs: torch.Tensor, inputs: Dict,  strategy: str = 'cls') -> np.ndarray:
    if strategy == 'cls':
        outputs = outputs[:, 0]
    elif strategy == 'mean':
        outputs = torch.sum(
            outputs * inputs["attention_mask"][:, :, None], dim=1) / torch.sum(inputs["attention_mask"])
    else:
        raise NotImplementedError
    return outputs.detach().cpu().numpy()

ModuleNotFoundError: No module named 'sentence_transformers'

In [10]:
docs = [
    transform_query('A man is eating a piece of bread'),
    "A man is eating food.",
    "A man is eating pasta.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
]

# 2. encode
inputs = tokenizer(docs, padding=True, return_tensors='pt')
for k, v in inputs.items():
    inputs[k] = v.cuda()
outputs = model(**inputs).last_hidden_state
embeddings = pooling(outputs, inputs, 'cls')

similarities = cos_sim(embeddings[0], embeddings[1:])
print('similarities:', similarities)

NameError: name 'transform_query' is not defined